In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np

In [2]:
from pathlib import Path
p = Path().cwd()
q = p/"Data"

images = [] # create an empty list
labels = [] # create an empty list

In [3]:
for fldr in q.iterdir(): 
    current_label = fldr.name # get the name of the current label 
    temp = q/current_label # here we are adding each if the inside folder name to the path so we can go through each of them
    for img in temp.iterdir():
      img = mpimg.imread(img) # load image
      #gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
      images.append(img) # add image to list of images 
      labels.append(fldr.name) # add label for this image to the list of labels 

UnidentifiedImageError: cannot identify image file 'C:\\Users\\alero\\Documents\\GitHub\\Stclairaro\\RecycleMates\\Data\\glass bottles\\Glass1,081.HEIC'